In [5]:
#from IPython.display import Image
#Image(filename='../resources/images/mp.gif')
from IPython.core.display import display, HTML
display(HTML('<img src="../resources/images/mp.gif">'))


In [1]:
import rosetta
import pyrosetta


pyrosetta.init('-ignore_unrecognized_res T')
pyrosetta.rosetta.basic.options.set_string_option('in:file:spanfile', '1bl8.span')

p = pyrosetta.pose_from_file('1bl8.nometal.pdb')

# setup pymol mover
pymolmover = pyrosetta.PyMolMover() #PyMOLMover()
pymolmover.keep_history(True)
pymolmover.apply(p)

## Add membrane to pose from span file
memadd = rosetta.protocols.membrane.AddMembraneMover()
memadd.spanfile('1bl8.span')
memadd.get_spanfile()

memadd.apply(p) # One of the membrane.util functions dies with metals (no CA for COM calculation)
pymolmover.apply(p)

## Step 1
## setup membrane position
setmempos = rosetta.protocols.membrane.SetMembranePositionMover()
setmempos.apply(p)
pymolmover.apply(p)

### Step 2
## transform protein into membrane
transformintomem = rosetta.protocols.membrane.TransformIntoMembraneMover()
transformintomem.use_default_membrane( True )
transformintomem.apply(p)
pymolmover.apply(p)

### Step 3
## optimize the position of the protein in the membrane (rotations)
optproteinpos = rosetta.protocols.membrane.OptimizeProteinEmbeddingMover()
optproteinpos.apply(p)
pymolmover.apply(p)

## Create a potassium ion residue
restypes = rosetta.core.chemical.ChemicalManager.get_instance().residue_type_set("fa_standard")
potassium = restypes.get_representative_type_name3("K  ")
confk = rosetta.core.conformation.Residue( potassium, True)

# see what jump to attach it after
#print p.num_jump()

## add the potassium residue by jump, so that it's DOF's are independent of the rest of the protein
p.append_residue_by_jump(confk,5)
pymolmover.apply(p)

## move the potassiumn to the center of mass of protein
comxyz = rosetta.core.pose.center_of_mass(p,1,p.total_residue())
p.set_xyz( rosetta.core.id.AtomID(1,390), comxyz )
pymolmover.apply(p)

# grab a sfxn
sfxn = pyrosetta.get_fa_scorefxn()



Found rosetta database at: /home/steve/anaconda2/lib/python2.7/site-packages/pyrosetta-4.0-py2.7.egg/database; using it....
PyRosetta-4 2016 [Rosetta 2016 unknown:7ce9d5bf5174d497560ce1624761e0c166dc30c3 2016-07-22 16:07:06 -0700] retrieved from: git@github.com:RosettaCommons/main.git
(C) Copyright Rosetta Commons Member Institutions.
Created in JHU by Sergey Lyskov and PyRosetta Team.



In [2]:
# Now in this loop, we will step down 1 A at a time, setup a test pose,
# use the neighbor selector to select residues within 10A to repack
# freeze everything else, and apply the  repack operation
for i in range(0,10):
    #print i
    # step down in Z direction
    step_z_xyz = rosetta.numeric.xyzVector_double_t(0,0,i)
    p.set_xyz( rosetta.core.id.AtomID(1,390), comxyz + step_z_xyz )
    
    # copy into testpose
    test_pose = rosetta.core.pose.Pose()
    test_pose.assign(p)
    sfxn(test_pose) # update neighbor graphs, you must do this before using nbr selector
    pymolmover.apply(test_pose)

    ## Get a packer task
    task = pyrosetta.standard_packer_task(test_pose)
    ## turn design off (it is on by default)
    task.restrict_to_repacking()
    nbrselect = rosetta.core.select.residue_selector.NeighborhoodResidueSelector()
    nbrselect.set_focus('390')
    nbrselect.set_distance(10.0) 

    ## get the inverse of the neighbor selection (everything else, we want to freeze these)
    notnbr =rosetta.core.select.residue_selector.NotResidueSelector(nbrselect)    
    nopackTO = rosetta.core.pack.task.operation.PreventRepackingRLT()
    op = rosetta.core.pack.task.operation.OperateOnResidueSubset( nopackTO, notnbr)
    ## Operate on our packer task with this operation
    op.apply(test_pose,task)

    # in case you want to visually look at the packer task
#    print  "Residue No.    DesignOn?    PackingOn?"
#     #for i in xrange(1,test_pose.total_residue()+1):
#     #    print i,task.being_designed(i),task.being_packed(i)

#   print sfxn(test_pose) ## get score before repacking
    # dump the pose after the metal is moved and before it is repacked
#    test_pose.dump_pdb('pre_pack_step_%s.pdb' %str(i))
    packmem = rosetta.protocols.simple_moves.PackRotamersMover(sfxn,task)
    packmem.apply(test_pose)

    #   print sfxn(test_pose) ## get score after repacking
    # dump pdb after packing to look at the difference
    #    test_pose.dump_pdb('post_pack_step_%s.pdb' %str(i))
    
    pymolmover.apply(test_pose)


0
1
2
3
4
5
6
7
8
9
